In [45]:
import numpy as np
import pandas as pd

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from num2words import num2words
from collections import Counter
from scipy import spatial
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from scipy.special import gammaln
from collections import Counter
from textblob import TextBlob

import operator
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle

In [51]:
stop_words = stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
def convert_numbers(k):
    for i in range(len(k)):
        try:
            num2words(int(k[i]))
            k[i] = " "
        except:
            pass
    return k
def preprocess(pd):
    pd = pd.str.lower()
    pd = pd.str.replace('[{}]'.format('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n\t'), ' ')
    pd = pd.apply(lambda x: [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(x)])
    pd = pd.apply(lambda x: convert_numbers(x))
    pd = pd.str.join(' ')
    pd = pd.str.replace('[{}]'.format(string.punctuation), ' ')
    
    pd = pd.apply(lambda x: [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(x)])    
    pd = pd.apply(lambda x: [item for item in x if item not in stop_words])
    return pd

In [52]:
dataset = pd.read_excel("OCR Output.xlsx")

In [56]:
dataset[6] = preprocess(dataset["Text"])
dataset[7] = dataset[6].apply(lambda x: " ".join(x))

In [57]:
data = dataset[7].values
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data)
vocab = vectorizer.get_feature_names()

In [58]:
k = 5
n_top_words = 10

In [59]:
model = LatentDirichletAllocation(n_topics=k, random_state=100)

In [60]:
id_topic = model.fit_transform(X)

/root/miniconda3/envs/python2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [61]:
topic_words = {}
for topic, comp in enumerate(model.components_):
    word_idx = np.argsort(comp)[::-1][:n_top_words]
    topic_words[topic] = [vocab[i] for i in word_idx]

In [62]:
for topic, words in topic_words.items():
    print('Topic: %d' % topic, ', '.join(words))

('Topic: 0', u'discharge, case, hospital, day, tab, wa, patient, admitted, name, date')
('Topic: 1', u'discharge, wa, case, tab, lad, mg, dr, consultant, day, bd')
('Topic: 2', u'discharge, hospital, dr, admission, case, date, day, vitals, treatment, patient')
('Topic: 3', u'discharge, hospital, name, case, day, dr, tab, date, time, summary')
('Topic: 4', u'discharge, case, tab, summary, mg, dr, wa, daily, severe, consultant')
